In [5]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import playerfantasyprofile
from nba_api.stats.static import players
import time
import pandas as pd

# Access NBA API via NBApy packages.
-- players.get_active_players: dictionary of players and ids

-- Create list of dataframes of each player's game logs using a loop. 

-- The game logs are generated by playergamelog.PlayerGameLog(). 

-- The get_data_frames()  call retrieves the game log as a dataframe wrapped in a list, so access with hash 0.

-- pd.concat() concatenates the new list of player gamelog dfs into one big df

-- time.sleep(0.600) allows the call not to timeout; however, I don't want to wait 5 minutes every time I work on this, so I'm saving the df to a csv, which I'll access hereafter and comment this loop portion out. Of course, I can do this because the NBA season is over. Mid-seaason, I'd have to pull the data again each day.

In [ ]:
player_dict = players.get_active_players()

In [56]:
# plgs = []
# for i in range(len(player_dict)):
#     pid = player_dict[i]['id']
#     plog = playergamelog.PlayerGameLog(player_id = pid,
#                                           season = '2020-21',
#                                           season_type_all_star = 'Regular Season')
#     plogdf = plog.get_data_frames()[0]
#     time.sleep(0.600)
#     plgs.append(plogdf)
# plogs21 = pd.concat(plgs)
# plogs21.to_csv('player_logs_2021', index = False)

# Prepare player logs dataframe (plogs)

-- read in CSV (saved from earlier code)

-- create new dictionary from player_dict that just has player ids as keys and full names as values

-- use id_dict to add player names column to plogs

In [77]:
plogs = pd.read_csv('player_logs_2021')

id_dict = {}
for d in player_dict:
    new_key = d['id']
    id_dict[new_key] = d['full_name']
    
plogs['PLAYER'] = plogs['Player_ID'].map(id_dict)